#  NREL-rex for NSRDB from Eagle and PYSAM simulation

There is an example of calling NSRDB and pysam on https://github.com/NREL/pysam/blob/master/Examples/FetchResourceFileExample.py

The difference is that example downloads the data into a .csv and saves the path file into the `solar_resource_file`.
When having access to Eagle directly, all the data is loaded dynamically from the `.h5` file so saving .csvs slows process, so I want to pass the weather data directly.


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
from rex import NSRDBX
import matplotlib.pyplot as plt

# NSRDBX

In [2]:
#TMY data located on eagle about 900GB
nsrdb_file = '/datasets/NSRDB/current/nsrdb_tmy-2021.h5'

In [3]:
#Input
region = 'Boulder'
region_col = 'county'
parameters = ['air_temperature', 'wind_speed', 'dhi', 'ghi', 'dni', 'surface_albedo']

In [4]:
#Load time and geographical infos
with NSRDBX(nsrdb_file, hsds=False) as f:
    # Get time index
    times = f.time_index
    # Get geographical index for region of interest
    gids = f.region_gids(region=region, region_col=region_col)   
    # Get meta data
    meta = f.meta[f.meta.index.isin(gids)]

In [5]:
#Load weather data
data = []
with NSRDBX(nsrdb_file, hsds=False) as f:
        for p in parameters:
            data.append(f.get_gid_df(p, gids)) #.values

In [6]:
#Create multi-level dataframe
columns = pd.MultiIndex.from_product([parameters, gids], names=["par", "gid"])
df_weather = pd.concat(data, axis=1)
df_weather.columns = columns
df_weather = df_weather.swaplevel(axis=1).sort_index(axis=1)

In [7]:
import pytz

In [8]:
#Create results dataframe
df_res = meta.loc[:, ['latitude', 'longitude']]
df_res['distance'] = np.nan

#loop through dataframe and perform computation 
#at the moment just saving the last location in the county of Boulder for use with pysam
for gid, row in meta.iterrows():
    meta_dict = row.to_dict()
    df_weather_gid = df_weather.loc[:, gid]
    tz_convert_val = meta_dict['timezone']
    df_weather_gid = df_weather_gid.tz_convert(pytz.FixedOffset(tz_convert_val*60))
    #df_weather_gid = df_weather_gid.tz_convert('Etc/GMT+7') # Localizing Data
    # Maping to 2021 because the localizing sets the first values to the year before december...
    df_weather_gid.index =  df_weather_gid.index.map(lambda t: t.replace(year=2021)) 
    # Then rearranging so they are at the end of hte dataframe, becuase I think SAM expect a 8760 starting at Jan 1 0 hours.
    df_weather_gid = df_weather_gid.sort_index()

In [9]:
# Sanity checks
print("MetaData: ", meta_dict)
print("\nWeather DF keys", df_weather_gid.keys())
print("\nDF length", len(df_weather_gid))
df_weather_gid.head(24)

MetaData:  {'latitude': 39.970001220703125, 'longitude': -105.05999755859375, 'elevation': 1614, 'timezone': -7, 'country': 'United States', 'state': 'Colorado', 'county': 'Boulder'}

Weather DF keys Index(['air_temperature', 'dhi', 'dni', 'ghi', 'surface_albedo', 'wind_speed'], dtype='object', name='par')

DF length 8760


par,air_temperature,dhi,dni,ghi,surface_albedo,wind_speed
time_index,,,,,,
2021-01-01 00:30:00-07:00,-13.8,0,0,0,0.8,5.2
2021-01-01 01:30:00-07:00,-13.7,0,0,0,0.8,5.3
2021-01-01 02:30:00-07:00,-13.6,0,0,0,0.8,5.3
2021-01-01 03:30:00-07:00,-13.5,0,0,0,0.8,5.2
2021-01-01 04:30:00-07:00,-13.5,0,0,0,0.8,5.1
2021-01-01 05:30:00-07:00,-13.5,0,0,0,0.8,4.9
2021-01-01 06:30:00-07:00,-13.5,0,0,0,0.8,4.8
2021-01-01 07:30:00-07:00,-12.6,0,0,0,0.8,4.9
2021-01-01 08:30:00-07:00,-10.7,16,0,16,0.8,5.3


In [10]:
df_weather_gid.tail(24)

par,air_temperature,dhi,dni,ghi,surface_albedo,wind_speed
time_index,,,,,,
2021-12-31 00:30:00-07:00,-5.1,0,0,0,0.2,1.6
2021-12-31 01:30:00-07:00,-5.1,0,0,0,0.2,1.2
2021-12-31 02:30:00-07:00,-5.1,0,0,0,0.2,0.5
2021-12-31 03:30:00-07:00,-5.4,0,0,0,0.2,0.4
2021-12-31 04:30:00-07:00,-5.9,0,0,0,0.2,0.7
2021-12-31 05:30:00-07:00,-6.2,0,0,0,0.2,1.2
2021-12-31 06:30:00-07:00,-6.0,0,0,0,0.2,1.9
2021-12-31 07:30:00-07:00,-4.6,0,0,0,0.2,2.5
2021-12-31 08:30:00-07:00,-1.5,32,698,151,0.2,2.6


## PySAM part

In [11]:
import PySAM.Pvsamv1 as PV
import PySAM.Grid as Grid
import PySAM.Utilityrate5 as UtilityRate
import PySAM.Cashloan as Cashloan
import pathlib
import json
import os

sif2 = 'AgriPV_SAMJsons'

import PySAM
PySAM.__version__

'4.0.0'

In [12]:
file_names = ["pvsamv1", "grid", "utilityrate5", "cashloan"]

pv2 = PV.new()  # also tried PVWattsSingleOwner
grid2 = Grid.from_existing(pv2)
ur2 = UtilityRate.from_existing(pv2)
so2 = Cashloan.from_existing(grid2, 'FlatPlatePVCommercial')



In [13]:
for count, module in enumerate([pv2, grid2, ur2, so2]):
    filetitle= 'AgriPV_SAM' + '_' + file_names[count] + ".json"
    with open(filetitle, 'r') as file:
        data = json.load(file)
        for k, v in data.items():
            if k == 'number_inputs':
                continue
            try:
                module.value(k, v)
            except AttributeError:
                # there is an error is setting the value for ppa_escalation
                print(module, k, v)

In [ ]:
pv2.SolarResource.solar_resource_file


In [ ]:
#Modifying it to a file that is local just for testing the results
pv2.SolarResource.solar_resource_file = 'phoenix_az_33.450495_-111.983688_psmv3_60_tmy.csv'

In [ ]:
#generator.SolarResource.assign({'solar_resource_file': nsrdb_fp})
#set_resource_data()
grid2.SystemOutput.gen = [0] * 8760  # p_out   # let's set all the values to 0
pv2.execute()
grid2.execute()
ur2.execute()
so2.execute()

results = pv2.Outputs.export()


In [14]:
# Sanity check 
dn = results['dn'][0:24]
dhi = results['df'][0:24]
alb = results['alb'][0:24]
poa = results['poa_front'][0:24]
power = results['subarray1_dc_gross'][0:24]
pd.DataFrame(zip(dn, dhi, alb, poa, power), columns=['dn', 'dhi', 'alb', 'poa', 'power'])

NameError: name 'results' is not defined

In [15]:
# meta.iloc[0]

['air_temperature', 'alpha', 'aod', 'asymmetry', 'cld_opd_dcomp', 'cld_press_acha', 'cld_reff_dcomp', 'clearsky_dhi', 'clearsky_dni', 'clearsky_ghi', 'cloud_fill_flag', 'cloud_type', 'dew_point', 'dhi', 'dni', 'fill_flag', 'ghi', 'meta', 'ozone', 'relative_humidity', 'solar_zenith_angle', 'ssa', 'surface_albedo', 'surface_pressure', 'time_index', 'tmy_year', 'tmy_year_short', 'total_precipitable_water', 'wind_direction', 'wind_speed']

In [16]:
pv2.SolarResource.solar_resource_file = ''

In [17]:
pv2.SolarResource.replace('solar_resource_file') # Removes the file so it doesnt try to read it.

In [18]:
# Sanity check , This should give an exception:
#pv2.SolarResource.solar_resource_file

In [19]:
pv2.SolarResource.assign({'solar_resource_data':{'dn':list(df_weather_gid.dni),
                                                   'df':list(df_weather_gid.dhi),
                                                   'tdry':list(df_weather_gid.air_temperature),
                                                   'wspd':list(df_weather_gid.wind_speed),
                                                   'lat':meta_dict['latitude'],
                                                   'lon':meta_dict['longitude'],
                                                   'tz':meta_dict['timezone'],
                                                   'Elevation':meta_dict['elevation'],
                                                   'Year':list(df_weather_gid.index.year),
                                                   'Month':list(df_weather_gid.index.month),
                                                   'Day':list(df_weather_gid.index.day),
                                                   'Hour':list(df_weather_gid.index.hour),
                                                   'Minute':list(df_weather_gid.index.minute),
                                                   'alb':list(df_weather_gid.surface_albedo)}})

In [20]:
# Making sure correct parameters are set on these 
pv2.SolarResource.irrad_mode = 0     # 0 for DNI and DHI input use
pv2.SolarResource.use_wf_albedo = 1  # 1 for using weather file albedo

In [21]:
# Assign albedo otherwise it fails even if use_wf_albedo=1, 'bug' reported
pv2.SolarResource.albedo = list(df_weather_gid.surface_albedo) # 
pv2.SolarResource.albedo = [0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2]

In [22]:
# help(pv2.SolarResource)

In [23]:
grid2.SystemOutput.gen = [0] * 8760  # p_out   # let's set all the values to 0
pv2.execute()
grid2.execute()
ur2.execute()
so2.execute()

results = pv2.Outputs.export()


In [24]:
# Sanity check 
dn = results['dn'][0:24]
dhi = results['df'][0:24]
alb = results['alb'][0:24]
poa = results['subarray1_poa_front'][0:24]
power = results['subarray1_dc_gross'][0:24]
pd.DataFrame(zip(dn, dhi, alb, poa, power), columns=['dn', 'dhi', 'alb', 'poa', 'power'])

,dn,dhi,alb,poa,power
0,0.0,0.0,0.8,0.000000,0.0
1,0.0,0.0,0.8,0.000000,0.0
2,0.0,0.0,0.8,0.000000,0.0
3,0.0,0.0,0.8,0.000000,0.0
4,0.0,0.0,0.8,0.000000,0.0
5,0.0,0.0,0.8,0.000000,0.0
6,0.0,0.0,0.8,0.000000,0.0
7,0.0,0.0,0.8,0.000000,0.0
8,0.0,16.0,0.8,13.363913,0.0
9,0.0,38.0,0.8,31.858243,0.0


In [26]:
df_weather_gid.dni.head(24)

time_index
2021-01-01 00:30:00-07:00      0
2021-01-01 01:30:00-07:00      0
2021-01-01 02:30:00-07:00      0
2021-01-01 03:30:00-07:00      0
2021-01-01 04:30:00-07:00      0
2021-01-01 05:30:00-07:00      0
2021-01-01 06:30:00-07:00      0
2021-01-01 07:30:00-07:00      0
2021-01-01 08:30:00-07:00      0
2021-01-01 09:30:00-07:00      0
2021-01-01 10:30:00-07:00      0
2021-01-01 11:30:00-07:00      0
2021-01-01 12:30:00-07:00     10
2021-01-01 13:30:00-07:00    513
2021-01-01 14:30:00-07:00    922
2021-01-01 15:30:00-07:00    790
2021-01-01 16:30:00-07:00    408
2021-01-01 17:30:00-07:00      0
2021-01-01 18:30:00-07:00      0
2021-01-01 19:30:00-07:00      0
2021-01-01 20:30:00-07:00      0
2021-01-01 21:30:00-07:00      0
2021-01-01 22:30:00-07:00      0
2021-01-01 23:30:00-07:00      0
Name: dni, dtype: uint16

In [ ]:
plt.plot(results['subarray1_dc_gross'])

In [ ]:
# results.keys()

In [ ]:
# Eventually will save and parse results from
# results['subarray1_ground_rear_spatial']